In [166]:
# pip als Paketmanager
#! pip install -q pyscipopt
#! pip install pandas
#!pip install openpyxl

In [167]:
from pyscipopt import Model, quicksum

# Optimierungsmodell zur Produktionsplanung

In [168]:
# Erstellen einer Modellinstanz
scip = Model()

## Indexmenge

In [169]:
H = [n for n in range(1, 25)]

## Batterie-Systemspezifikationen

In [170]:
fixe_zykluskosten = False

wirkungsgrad_wechselrichter = 0.985
wirkungsgrad_laden = 0.975
round_trip_efficiency = 0.95
entlade_verlust = wirkungsgrad_laden - round_trip_efficiency

wirkungsgrad_systemeingang = wirkungsgrad_wechselrichter * wirkungsgrad_laden
wirkungsgrad_systemausgang = (1-(entlade_verlust / wirkungsgrad_laden)) * wirkungsgrad_wechselrichter


f_e = wirkungsgrad_systemeingang # Faktor Einkauf
f_v = wirkungsgrad_systemausgang # Faktor Verkauf


nennkapazität = 40 # MWh brutto
lademinimum = 0.2 # 20%
lademaximum = 1 # 100%
anfangsbestand = 0.5 # 50%

nettokapazität = zyklus = nennkapazität * (lademaximum - lademinimum) # MWh netto
zykluskosten = 1500 # € / zyklus
mwh_zykluskosten = zykluskosten / zyklus # € / MWh

erlaubte_zyklen_pro_tag = 2

a = anfangsbestand * nennkapazität # MWh Anfangs- und Endbestand
u = lademinimum * nennkapazität # MWh Untergrenze Batteriekapazität
o = lademaximum * nennkapazität # MWh Obergrenze Batteriekapazität 

c = 0.5 # nennkapazität / h


# Sicherstellen, dass unsere Faktoren für Systemeingang und -ausgang den multiplizierten Wirkungsgraden entspricht
print(wirkungsgrad_systemeingang*wirkungsgrad_systemausgang)
print(wirkungsgrad_wechselrichter*wirkungsgrad_wechselrichter*round_trip_efficiency)



0.92171375
0.92171375


## Vorhersagedaten

In [171]:
import pandas as pd

prognose = pd.read_excel('Preisprognosen.xlsx')

p_h = {}

for h in H:
    stundenprognose = prognose[prognose['Stunde'] == h]
    checksum = stundenprognose['Wahrscheinlichkeit'].sum()
    if checksum != 1:
        print('WARNING: Prognosedaten unvollständig')

    erwartungswert_strompreis = (stundenprognose['Strompreis'] * stundenprognose['Wahrscheinlichkeit']).sum() / checksum
    p_h[h] = erwartungswert_strompreis

print (p_h)

{1: np.float64(81.52000000000001), 2: np.float64(72.24000000000001), 3: np.float64(68.16000000000001), 4: np.float64(66.64), 5: np.float64(66.8), 6: np.float64(71.04), 7: np.float64(88.55999999999999), 8: np.float64(92.68), 9: np.float64(80.92), 10: np.float64(61.080000000000005), 11: np.float64(43.080000000000005), 12: np.float64(30.880000000000003), 13: np.float64(22.44), 14: np.float64(15.959999999999999), 15: np.float64(12.52), 16: np.float64(18.44), 17: np.float64(34.88), 18: np.float64(56.56), 19: np.float64(78.44000000000001), 20: np.float64(105.60000000000001), 21: np.float64(144.8), 22: np.float64(125.12), 23: np.float64(101.36), 24: np.float64(85.28)}


## Entscheidungsvariablen

In [172]:
e_h={}
v_h={}

for h in H:
    e_h[h] = scip.addVar(vtype='C', lb=0, ub=None, name=f"e_{h}")
    v_h[h] = scip.addVar(vtype='C', lb=0, ub=None, name=f"v_{h}")

print('Entscheidungsvariablen =', scip.getVars())

Entscheidungsvariablen = [e_1, v_1, e_2, v_2, e_3, v_3, e_4, v_4, e_5, v_5, e_6, v_6, e_7, v_7, e_8, v_8, e_9, v_9, e_10, v_10, e_11, v_11, e_12, v_12, e_13, v_13, e_14, v_14, e_15, v_15, e_16, v_16, e_17, v_17, e_18, v_18, e_19, v_19, e_20, v_20, e_21, v_21, e_22, v_22, e_23, v_23, e_24, v_24]


## Zielfunktion

In [173]:
gewinn_kauf_verkauf = quicksum((p_h[h] * (v_h[h] - e_h[h])) for h in H)

if fixe_zykluskosten:
    zykluskosten = 3000
else:
    zykluskosten = quicksum((e_h[h] * f_e * mwh_zykluskosten) for h in H)

scip.setObjective(gewinn_kauf_verkauf - zykluskosten, sense="maximize")
print(scip.getObjective())

Expr({Term(e_1): -126.53757812500001, Term(v_1): 81.52000000000001, Term(e_2): -117.25757812500001, Term(v_2): 72.24000000000001, Term(e_3): -113.17757812500001, Term(v_3): 68.16000000000001, Term(e_4): -111.657578125, Term(v_4): 66.64, Term(e_5): -111.817578125, Term(v_5): 66.8, Term(e_6): -116.057578125, Term(v_6): 71.04, Term(e_7): -133.577578125, Term(v_7): 88.55999999999999, Term(e_8): -137.697578125, Term(v_8): 92.68, Term(e_9): -125.937578125, Term(v_9): 80.92, Term(e_10): -106.09757812500001, Term(v_10): 61.080000000000005, Term(e_11): -88.09757812500001, Term(v_11): 43.080000000000005, Term(e_12): -75.897578125, Term(v_12): 30.880000000000003, Term(e_13): -67.457578125, Term(v_13): 22.44, Term(e_14): -60.977578125, Term(v_14): 15.959999999999999, Term(e_15): -57.537578124999996, Term(v_15): 12.52, Term(e_16): -63.457578125, Term(v_16): 18.44, Term(e_17): -79.897578125, Term(v_17): 34.88, Term(e_18): -101.577578125, Term(v_18): 56.56, Term(e_19): -123.45757812500001, Term(v_19)

## Nebenbedingungen / Restriktionen

In [174]:
# Ladestand zur Stunde 0 = Ladestand zur Stunde 24, also Summe Lademenge und Entlademenge gleich
scip.addCons(quicksum(((e_h[h] * f_e) - (v_h[h] / f_v)) for h in H) == 0, name="Anfangs- und Endbestand gleich") 

# Maximale Ladezyklen am pro Tag anhand der Einkaufsmenge (mit Faktor = Lademenge), alternativ anhand der Verkaufsmenge
scip.addCons(quicksum((e_h[h] * f_e) for h in H) <= (erlaubte_zyklen_pro_tag * nettokapazität), name="Maximale Ladezyklen pro Tag")

# Mindestladestand nicht unterschritten und Höchstladestand nicht überschritten
for h in H:
    H_t =  [n for n in range(1, h+1)]
    scip.addCons( (a + quicksum(((e_h[t] * f_e) - (v_h[t] / f_v)) for t in H_t)) >= u, name=f"Mindestladestand zum Zeitpunkt t={h}")
    scip.addCons( (a + quicksum(((e_h[t] * f_e) - (v_h[t] / f_v)) for t in H_t)) <= o, name=f"Maximalladestand zum Zeitpunkt t={h}")

# Lade- und Entladeleistung begrenzt (C-Rate)
for h in H:
    scip.addCons((e_h[h] * f_e) + (v_h[h] / f_v) <= c * nennkapazität, name=f"Lade-/Entladeleistung der Stunde h={h}")

print('Nebenbedingungen =', scip.getConss())

Nebenbedingungen = [Anfangs- und Endbestand gleich, Maximale Ladezyklen pro Tag, Mindestladestand zum Zeitpunkt t=1, Maximalladestand zum Zeitpunkt t=1, Mindestladestand zum Zeitpunkt t=2, Maximalladestand zum Zeitpunkt t=2, Mindestladestand zum Zeitpunkt t=3, Maximalladestand zum Zeitpunkt t=3, Mindestladestand zum Zeitpunkt t=4, Maximalladestand zum Zeitpunkt t=4, Mindestladestand zum Zeitpunkt t=5, Maximalladestand zum Zeitpunkt t=5, Mindestladestand zum Zeitpunkt t=6, Maximalladestand zum Zeitpunkt t=6, Mindestladestand zum Zeitpunkt t=7, Maximalladestand zum Zeitpunkt t=7, Mindestladestand zum Zeitpunkt t=8, Maximalladestand zum Zeitpunkt t=8, Mindestladestand zum Zeitpunkt t=9, Maximalladestand zum Zeitpunkt t=9, Mindestladestand zum Zeitpunkt t=10, Maximalladestand zum Zeitpunkt t=10, Mindestladestand zum Zeitpunkt t=11, Maximalladestand zum Zeitpunkt t=11, Mindestladestand zum Zeitpunkt t=12, Maximalladestand zum Zeitpunkt t=12, Mindestladestand zum Zeitpunkt t=13, Maximallades

## Berechnung der Lösung

In [175]:
scip.setIntParam("display/verblevel", 5)  # Set verbosity level to 5


scip.optimize()
# Status des Solvers
status = scip.getStatus()
print(f"Status des Solvers: {status} \n")

if status == "optimal":
    print('LÖSUNG:')
    print('Zielfunktionswert (Gewinn) =', scip.getObjVal())
    for h in H:
        print("EINKAUF Stunde", h, " : " , scip.getVal(e_h[h]))
        print("Verkauf Stunde", h, " : " , scip.getVal(v_h[h]))

else:
    print('Problem hat keine Lösung')



LP Solver <Soplex 7.1.1>: barrier convergence tolerance cannot be set -- tolerance of SCIP and LP solver may differ
LP Solver <Soplex 7.1.1>: fastmip setting not available -- SCIP parameter has no effect
LP Solver <Soplex 7.1.1>: number of threads settings not available -- SCIP parameter has no effect
Status des Solvers: optimal 

LÖSUNG:
Zielfunktionswert (Gewinn) = 1886.6522808278019
EINKAUF Stunde 1  :  0.0
Verkauf Stunde 1  :  0.0
EINKAUF Stunde 2  :  0.0
Verkauf Stunde 2  :  0.0
EINKAUF Stunde 3  :  0.0
Verkauf Stunde 3  :  0.0
EINKAUF Stunde 4  :  0.0
Verkauf Stunde 4  :  0.0
EINKAUF Stunde 5  :  0.0
Verkauf Stunde 5  :  0.0
EINKAUF Stunde 6  :  0.0
Verkauf Stunde 6  :  0.0
EINKAUF Stunde 7  :  0.0
Verkauf Stunde 7  :  0.0
EINKAUF Stunde 8  :  0.0
Verkauf Stunde 8  :  11.516923076923078
EINKAUF Stunde 9  :  0.0
Verkauf Stunde 9  :  -1.7048470897115737e-15
EINKAUF Stunde 10  :  0.0
Verkauf Stunde 10  :  0.0
EINKAUF Stunde 11  :  0.0
Verkauf Stunde 11  :  0.0
EINKAUF Stunde 12  :  

In [176]:
einkauf_sum = sum(scip.getVal(e_h[h]) for h in H)
print(f'Es wurden {einkauf_sum} MWh eingekauft')
verkauf_sum = sum(scip.getVal(v_h[h]) for h in H)
print(f'Es wurden {verkauf_sum} MWh verkauft')

gesamtwirkungsgrad = (0.95*0.985*0.985)


if(gesamtwirkungsgrad != verkauf_sum / einkauf_sum):
    print('CHECK: Wirkungsgrad stimmt nicht überein')
else:
    print('CHECK: Wirkungsgrad stimmt überein')

Es wurden 33.32031758427698 MWh eingekauft
Es wurden 30.711794871794872 MWh verkauft
CHECK: Wirkungsgrad stimmt nicht überein
